<a href="https://colab.research.google.com/github/DnYAlv/IMDB_sentiment_analysis/blob/main/IMDB_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 6.5 MB/s 
     |████████████████████████████████| 3.3 MB 57.3 MB/s 
     |████████████████████████████████| 636 kB 60.4 MB/s 
     |████████████████████████████████| 895 kB 70.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# TensorFlow
import tensorflow as tf
import tensorflow_datasets as tfds

# HuggingFace NLP library
from transformers import TFBertForSequenceClassification, BertTokenizer

# Model Evaluation
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import datetime

In [ ]:
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/DnYAlv/IMDB_sentiment_analysis/main/Documents/Bootcamp%20ML/AMLI/IMDB%20Dataset.csv/IMDB%20Dataset.csv")

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [ ]:
df.shape

(50000, 2)

In [ ]:
df['sentiment'] = df['sentiment'].astype('category').cat.codes
df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X = df['review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40000,)
(10000,)
(40000,)
(10000,)


In [ ]:
train = pd.DataFrame({'text': X_train, 'label': y_train})
test = pd.DataFrame({'text': X_test, 'label': y_test})

In [ ]:
train.to_csv('imdb_review_train.csv', index=False)
test.to_csv('imdb_review_test.csv', index=False)

In [ ]:
TRAIN_FILE_PATH = 'imdb_review_train.csv'
TEST_FILE_PATH = 'imdb_review_test.csv'

In [ ]:
ds_train = tf.data.experimental.CsvDataset([TRAIN_FILE_PATH],
                                           record_defaults=[tf.constant([""], dtype=tf.string), tf.constant([0], dtype=tf.int64)],
                                           header=True)

ds_test = tf.data.experimental.CsvDataset([TEST_FILE_PATH],
                                           record_defaults=[tf.constant([""], dtype=tf.string), tf.constant([0], dtype=tf.int64)],
                                           header=True)

In [ ]:
for message, label in ds_train.take(5):
    print(message, label)

tf.Tensor(b'I caught this little gem totally by accident back in 1980 or \'81. I was at a revival theatre to see two old silly sci-fi movies. The theatre was packed full and (with no warning) they showed a bunch of sci-fi short spoofs (to get us in the mood). Most were somewhat amusing but THIS came on and, within seconds, the audience was in hysterics! The biggest laugh came when they showed "Princess Laia" having huge cinnamon buns instead of hair on her head. She looks at the camera, gives a grim smile and nods. That made it even funnier! You gotta see "Chewabacca" played by what looks like a Muppet! It was extremely silly and stupid...but I couldn\'t stop laughing. Most of the dialogue was drowned out because of all the laughter. Also if you know "Star Wars" pretty well it\'s even funnier--they deliberately poke fun at some of the dialogue. This REALLY works with an audience! A definite 10!', shape=(), dtype=string) tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(b'I can\'t believe t

In [ ]:
CLASSES = ['Positive', 'Negative']

In [ ]:
MODEL_NAME = 'indobenchmark/indobert-lite-base-p2'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='indobenchmark/indobert-lite-base-p2', vocab_size=29999, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
# example_sentence = 'This Movie is outstandingly great'

In [ ]:
# bert_input = tokenizer.encode_plus(
#     example_sentence,
#     max_length=25, # max length of the text that can go to BERT
#     add_special_tokens=True, # add special tokens
#     pad_to_max_length=True, # add [PAD] tokens
#     return_attention_mask=True, # add attention mask so model won't focus on padded tokens
#     truncation=True # truncate to a maximum length specified by max_length
# )

# print(bert_input)

{'input_ids': [2, 5379, 15060, 374, 3009, 29183, 1290, 13332, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
MAX_LENGTH = 128

In [ ]:
def convert_sentence_to_features(sentence):
    return tokenizer.encode_plus(
        sentence,
        max_length=MAX_LENGTH,
        add_special_tokens=True,
        pad_to_max_length=True,
        return_attention_mask=True,
        truncation=True
    )

def map_features_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
        'input_ids': input_ids,
        'token_type_ids': token_type_ids,
        'attention_mask': attention_masks,
    }, label

def encode_sentences(dataset):
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []

    for message, label in tfds.as_numpy(dataset):
        bert_input = convert_sentence_to_features(message.decode())
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])

    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_features_to_dict)

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BATCH_SIZE = 10000

In [ ]:
ds_train_encoded = encode_sentences(ds_train).shuffle(SHUFFLE_BATCH_SIZE).batch(BATCH_SIZE)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
ds_test_encoded = encode_sentences(ds_test).batch(BATCH_SIZE) 

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
for message, label in ds_train_encoded.take(1):
    print(message, label)

{'input_ids': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
array([[    2,  5379,   374, ...,    46, 29948,     3],
       [    2,  6855,  1290, ...,     0,     0,     0],
       [    2,    89,  4081, ...,   490,  2246,     3],
       ...,
       [    2,    48,  1002, ..., 29943,   136,     3],
       [    2,  8790, 29935, ...,  1054,  1135,     3],
       [    2,    89,  7621, ..., 16669, 16051,     3]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>}

In [ ]:
LEARNING_RATE = 1e-6
EPOCHS = 10

In [ ]:
MODEL_NAME

'indobenchmark/indobert-lite-base-p2'

In [ ]:
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some layers from the model checkpoint at indobenchmark/indobert-lite-base-p2 were not used when initializing TFBertForSequenceClassification: ['predictions', 'albert', 'sop_classifier']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-lite-base-p2 and are newly initialized: ['bert', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

In [ ]:
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=[metric])

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109081344 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 109,083,651
Trainable params: 109,083,651
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%reload_ext tensorboard
log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, profile_batch=0) 
tensorboard_callback

In [34]:
history = model.fit(ds_train_encoded,
                    epochs=EPOCHS,
                    validation_data=ds_test_encoded,
                    callbacks=[tensorboard_callback])

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autog